# Performance Evaluation

In [3]:
import json
import torch
import numpy as np
import sys
import os

from tqdm import tqdm

from rdkit import Chem, RDLogger

# Shut up RDKit
logger = RDLogger.logger()
logger.setLevel(RDLogger.CRITICAL)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Draw

import random

import importlib



In [4]:
sys.path.append('./SD_LSTM/')
sys.path.append('./LSTM_TF/')
sys.path.append('./VANILLA_VAE/')
sys.path.append('./SD_VAE/')

from sd_vae_sampler import SDVAESampler
from model_sd_vae import SDVAE
from sd_lstm_sampler import SDLSTMSampler
from sd_lstm_utils import load_model as load_sd_lstm_model

from benchmark_vanilla_vae import VanillaVAEHarness

from model_vanilla_vae import VanillaMolVAE
from rnn_utils import load_model

from fast_rnn_sampler import FastSampler
# from rnn_sampler import ConditionalSmilesRnnSampler
from rnn_utils import load_rnn_model


sys.path.append('utils/')

from smiles_char_dict import SmilesCharDictionary
sd = SmilesCharDictionary()

from evaluation_utils import absolute_metrics, amina_metrics, property_metrics, props_from_smiles, plot_smiles, benchmark_reconstruction_QM9

In [5]:
sys.path.append('../utils/')
from property_calculator import PropertyCalculator

pc = PropertyCalculator(['LogP'])

# Load Models

In [4]:
!ls models/NEW_LONG_RUNS/QM9/

EXP-cVAE-Pol   EXP-cVAE-TF10	 Old-cVAE-Pol  cVAE-Pol
EXP-cVAE-Pol2  Old-Neg-cVAE-Pol  cVAE-KLD      cVAE-Pol2


In [6]:
# Load Vanilla cVAE Model

# model_definit = 'models/LONG_RUNS/QM9/cVAE/SD_REG_VANILLA_VAE_quiet-dew-12_Epoch_420_Vl_0.115.json'
# model_weights = 'models/LONG_RUNS/QM9/cVAE/SD_REG_VANILLA_VAE_quiet-dew-12_Epoch_420_Vl_0.115.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/cVAE-KLD/SD_REG_VANILLA_VAE_ancient-paper-11_Epoch_235_Vl_0.353.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/cVAE-KLD/SD_REG_VANILLA_VAE_ancient-paper-11_Epoch_235_Vl_0.353.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/Old-cVAE-Pol/SD_REG_VANILLA_VAE_delicate-queen-93_Epoch_195_Vl_0.043.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/Old-cVAE-Pol/SD_REG_VANILLA_VAE_delicate-queen-93_Epoch_195_Vl_0.043.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/Old-Neg-cVAE-Pol/SD_REG_VANILLA_VAE_ancient-tooth-26_Epoch_190_Vl_0.713.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/Old-Neg-cVAE-Pol/SD_REG_VANILLA_VAE_ancient-tooth-26_Epoch_190_Vl_0.713.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/cVAE-Pol/SD_REG_VANILLA_VAE_young-king-99_Epoch_381_Vl_0.165.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/cVAE-Pol/SD_REG_VANILLA_VAE_young-king-99_Epoch_381_Vl_0.165.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/cVAE-Pol/SD_REG_VANILLA_VAE_young-king-99_Epoch_150_Vl_0.169.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/cVAE-Pol/SD_REG_VANILLA_VAE_young-king-99_Epoch_150_Vl_0.169.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/cVAE-Pol/SD_REG_VANILLA_VAE_young-king-99_Epoch_50_Vl_0.262.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/cVAE-Pol/SD_REG_VANILLA_VAE_young-king-99_Epoch_50_Vl_0.262.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/cVAE-Pol2/SD_REG_VANILLA_VAE_still-rain-34_Epoch_313_Vl_0.349.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/cVAE-Pol2/SD_REG_VANILLA_VAE_still-rain-34_Epoch_313_Vl_0.349.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/Weighted-cVAE-Pol/SD_REG_VANILLA_VAE_lingering-sunset-29_Epoch_78_Vl_0.096.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/Weighted-cVAE-Pol/SD_REG_VANILLA_VAE_lingering-sunset-29_Epoch_78_Vl_0.096.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/Weighted-cVAE-Pol/SD_REG_VANILLA_VAE_lingering-sunset-29_Epoch_264_Vl_0.053.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/Weighted-cVAE-Pol/SD_REG_VANILLA_VAE_lingering-sunset-29_Epoch_264_Vl_0.053.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/Weighted-cVAE-Pol2/SD_REG_VANILLA_VAE_red-waterfall-24_Epoch_300_Vl_0.076.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/Weighted-cVAE-Pol2/SD_REG_VANILLA_VAE_red-waterfall-24_Epoch_300_Vl_0.076.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/cVAE/SD_REG_VANILLA_VAE_dawn-band-89_Epoch_294_Vl_0.086.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/cVAE/SD_REG_VANILLA_VAE_dawn-band-89_Epoch_294_Vl_0.086.pt'

# model_definit = 'models/NEW_LONG_RUNS/Uncond-QM9/cVAE/SD_REG_VANILLA_VAE_snowy-night-57_Epoch_260_Vl_0.052.json'
# model_weights = 'models/NEW_LONG_RUNS/Uncond-QM9/cVAE/SD_REG_VANILLA_VAE_snowy-night-57_Epoch_260_Vl_0.052.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/Weighted-cVAE-Pol/SD_REG_VANILLA_VAE_lingering-sunset-29_Epoch_78_Vl_0.096.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/Weighted-cVAE-Pol/SD_REG_VANILLA_VAE_lingering-sunset-29_Epoch_78_Vl_0.096.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/Weighted-cVAE-Pol/SD_REG_VANILLA_VAE_lingering-sunset-29_Epoch_120_Vl_0.070.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/Weighted-cVAE-Pol/SD_REG_VANILLA_VAE_lingering-sunset-29_Epoch_120_Vl_0.070.pt'

# model_definit = 'models/NEW_LONG_RUNS/Uncond-QM9/cVAE/SD_REG_VANILLA_VAE_green-firefly-95_Epoch_327_Vl_0.244.json'
# model_weights = 'models/NEW_LONG_RUNS/Uncond-QM9/cVAE/SD_REG_VANILLA_VAE_green-firefly-95_Epoch_327_Vl_0.244.pt'

# model_definit = 'models/NEW_LONG_RUNS/Uncond-QM9/cVAE/SD_REG_VANILLA_VAE_gentle-boat-25_Epoch_180_Vl_0.061.json'
# model_weights = 'models/NEW_LONG_RUNS/Uncond-QM9/cVAE/SD_REG_VANILLA_VAE_gentle-boat-25_Epoch_180_Vl_0.061.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/cVAE-KLD-16Samples/SD_REG_VANILLA_VAE_still-forest-60_Epoch_33_Vl_0.187.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/cVAE-KLD-16Samples/SD_REG_VANILLA_VAE_still-forest-60_Epoch_33_Vl_0.187.pt'

cvae_sampler = VanillaVAEHarness(batch_size=64, device='cpu')

cvae_model = load_model(model_class=VanillaMolVAE, model_definition=model_definit, model_weights=model_weights, device='cpu')

# ugly hack
cvae_model = cvae_model.to('cpu')
cvae_model.device = 'cpu'
cvae_model.encoder.device = 'cpu'
cvae_model.state_decoder.device = 'cpu'

a Conv1d inited
a Conv1d inited
a Conv1d inited
a Linear inited
a Linear inited
a Linear inited
a Linear inited
a GRU inited
a Linear inited


In [20]:
# Load Explicit cVAE Model

# TODO: FIX NAME CLASHES
file_path = 'ACTION_SAMPLING_VANILLA_VAE/model_vanilla_vae.py'

# Load the module specified by the file path
spec = importlib.util.spec_from_file_location("model_vanilla_vae.py", file_path)
module = importlib.util.module_from_spec(spec)
sys.modules["model_vanilla_vae"] = module
spec.loader.exec_module(module)

explicit_model_class = module.VanillaMolVAE

file_path = 'ACTION_SAMPLING_VANILLA_VAE/action_sampling_vae_sampler.py'

# Load the module specified by the file path
spec = importlib.util.spec_from_file_location("action_sampling_vae_sampler.py", file_path)
module = importlib.util.module_from_spec(spec)
sys.modules["action_sampling_vae_sampler"] = module
spec.loader.exec_module(module)

harness_class = module.VanillaVAEHarness

# model_definit = 'models/NEW_LONG_RUNS/QM9/EXP-cVAE-Pol2/SD_REG_VANILLA_VAE_shiny-sun-14_Epoch_170_Vl_0.244.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/EXP-cVAE-Pol2/SD_REG_VANILLA_VAE_shiny-sun-14_Epoch_170_Vl_0.244.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/EXP-cVAE-Pol/SD_REG_VANILLA_VAE_fragrant-math-98_Epoch_127_Vl_0.367.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/EXP-cVAE-Pol/SD_REG_VANILLA_VAE_fragrant-math-98_Epoch_127_Vl_0.367.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/EXP-cVAE-Pol2-TF10/SD_REG_VANILLA_VAE_weathered-meadow-69_Epoch_143_Vl_0.126.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/EXP-cVAE-Pol2-TF10/SD_REG_VANILLA_VAE_weathered-meadow-69_Epoch_143_Vl_0.126.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/EXP-cVAE-Pol/SD_REG_VANILLA_VAE_fragrant-math-98_Epoch_188_Vl_0.357.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/EXP-cVAE-Pol/SD_REG_VANILLA_VAE_fragrant-math-98_Epoch_188_Vl_0.357.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/EXP-cVAE-TF10/SD_REG_VANILLA_VAE_bold-bonus-90_Epoch_216_Vl_0.034.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/EXP-cVAE-TF10/SD_REG_VANILLA_VAE_bold-bonus-90_Epoch_216_Vl_0.034.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/EXP-cVAE-KLD-TF10/SD_REG_VANILLA_VAE_small-lake-03_Epoch_110_Vl_0.119.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/EXP-cVAE-KLD-TF10/SD_REG_VANILLA_VAE_small-lake-03_Epoch_110_Vl_0.119.pt'

# model_definit = 'models/LONG_RUNS/QM9/EXP-cVAE-KLD/SD_REG_VANILLA_VAE_quiet-tooth-22_Epoch_155_Vl_0.234.json'
# model_weights = 'models/LONG_RUNS/QM9/EXP-cVAE-KLD/SD_REG_VANILLA_VAE_quiet-tooth-22_Epoch_155_Vl_0.234.pt'

# model_definit = 'models/LONG_RUNS/QM9/EXP-cVAE-KLD-TF/SD_REG_VANILLA_VAE_raspy-wind-80_Epoch_228_Vl_0.224.json'
# model_weights = 'models/LONG_RUNS/QM9/EXP-cVAE-KLD-TF/SD_REG_VANILLA_VAE_raspy-wind-80_Epoch_228_Vl_0.224.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/EXP-cVAE-KLD-TF10/SD_REG_VANILLA_VAE_small-lake-03_Epoch_110_Vl_0.119.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/EXP-cVAE-KLD-TF10/SD_REG_VANILLA_VAE_small-lake-03_Epoch_110_Vl_0.119.pt'

# model_definit = 'models/NEW_LONG_RUNS/QM9/EXP-cVAE-TF10/SD_REG_VANILLA_VAE_bold-bonus-90_Epoch_216_Vl_0.034.json'
# model_weights = 'models/NEW_LONG_RUNS/QM9/EXP-cVAE-TF10/SD_REG_VANILLA_VAE_bold-bonus-90_Epoch_216_Vl_0.034.pt'

# model_definit = 'models/NEW_LONG_RUNS/Uncond-QM9/EXP-cVAE/SD_REG_VANILLA_VAE_summer-hall-94_Epoch_317_Vl_0.136.json'
# model_weights = 'models/NEW_LONG_RUNS/Uncond-QM9/EXP-cVAE/SD_REG_VANILLA_VAE_summer-hall-94_Epoch_317_Vl_0.136.pt'

# model_definit = 'models/NEW_LONG_RUNS/Uncond-QM9/EXP-cVAE-TF01/SD_REG_VANILLA_VAE_yellow-art-56_Epoch_259_Vl_0.107.json'
# model_weights = 'models/NEW_LONG_RUNS/Uncond-QM9/EXP-cVAE-TF01/SD_REG_VANILLA_VAE_yellow-art-56_Epoch_259_Vl_0.107.pt'

model_definit = 'models/NEW_LONG_RUNS/Uncond-QM9/EXP-cVAE-TF10/SD_REG_VANILLA_VAE_aged-forest-04_Epoch_314_Vl_0.027.json'
model_weights = 'models/NEW_LONG_RUNS/Uncond-QM9/EXP-cVAE-TF10/SD_REG_VANILLA_VAE_aged-forest-04_Epoch_314_Vl_0.027.pt'

# model_definit = 'models/LONG_RUNS/QM9/EXP-cVAE/SD_REG_VANILLA_VAE_billowing-darkness-52_Epoch_492_Vl_0.151.json'
# model_weights = 'models/LONG_RUNS/QM9/EXP-cVAE/SD_REG_VANILLA_VAE_billowing-darkness-52_Epoch_492_Vl_0.151.pt'

# model_definit = 'models/LONG_RUNS/QM9/EXP-cVAE/SD_REG_VANILLA_VAE_gentle-credit-23_Epoch_461_Vl_0.054.json'
# model_weights = 'models/LONG_RUNS/QM9/EXP-cVAE/SD_REG_VANILLA_VAE_gentle-credit-23_Epoch_461_Vl_0.054.pt'

# model_definit = 'models/LONG_RUNS/QM9/EXP-cVAE-TF/SD_REG_VANILLA_VAE_late-leaf-82_Epoch_421_Vl_0.112.json'
# model_weights = 'models/LONG_RUNS/QM9/EXP-cVAE-TF/SD_REG_VANILLA_VAE_late-leaf-82_Epoch_421_Vl_0.112.pt'

exp_cvae_sampler = harness_class(batch_size=64, device='cpu')

exp_cvae_model = load_model(model_class=explicit_model_class, model_definition=model_definit, model_weights=model_weights, device='cpu')


a Conv1d inited
a Conv1d inited
a Conv1d inited
a Linear inited
a Linear inited
a Linear inited
a GRU inited
a Linear inited


In [8]:
# sample(self, model, properties, random=True, latent_points = None)
# sd.decode()
sample_props = np.array( [[a] for a in random.choices(test_props, k=50)])
print(list(cvae_sampler.sample(cvae_model, sample_props)))

NameError: name 'test_props' is not defined

In [9]:
pro_sample = random.choices(test_props, )
smi_sample = random.choices(test_smiles, k=10)

print(smi_sample[0])

input_tokens = smiles_to_tokens(smi_sample, cvae_model.max_decode_steps)

print(input_tokens[:10][0])

encodings = cvae_model.encoder(input_tokens)[0]
samp_latents = cvae_model.reparameterize(mu = encodings, logvar = torch.tensor(cvae_model.eps_std))

out_actions = model.state_decoder(z, normd_props, x_inputs = None, teacher_forcing = False, return_logits = False)


NameError: name 'test_props' is not defined

# Load Dataset

In [10]:
## ZINC
dataset_path = "data/ZINC250K/"
qmds = pd.read_csv(os.path.join(dataset_path, 'ZINC_clean.csv'))
print(qmds.columns)
props = list(qmds.drop(['ZINC_ID', 'SMILES'], axis=1).columns)
print(f'Properties in dataset: {props}')

# Seperate Test Train and Validation Datasets
indecies = np.load('data/ZINC250K/data_splits.npy')

train_smiles = np.array(qmds['SMILES'])[indecies == 0]
val_smiles = np.array(qmds['SMILES'])[indecies == 1]
test_smiles = np.array(qmds['SMILES'])[indecies == 2]

test_props = [ float(a) for a in pd.read_csv('data/ZINC250K/ZINC_clean.csv')['LogP'][indecies == 2]]

Index(['ZINC_ID', 'SMILES', 'LogP'], dtype='object')
Properties in dataset: ['LogP']


In [11]:
## QM9
dataset_path = "data/QM9/"
qmds = pd.read_csv(os.path.join(dataset_path, 'QM9_clean.csv'))
print(qmds.columns)
props = list(qmds.drop(['QM9_id', 'SMILES'], axis=1).columns)
print(f'Properties in dataset: {props}')

# Seperate Test Train and Validation Datasets
indecies = np.load('data/QM9/data_splits.npy')

train_smiles = np.array(qmds['SMILES'])[indecies == 0]
val_smiles = np.array(qmds['SMILES'])[indecies == 1]
test_smiles = np.array(qmds['SMILES'])[indecies == 2]

test_props = [ float(a) for a in pd.read_csv('data/QM9/QM9_clean.csv')['LogP'][indecies == 2]]

Index(['QM9_id', 'SMILES', 'LogP'], dtype='object')
Properties in dataset: ['LogP']


# Metric Computation

### TODO: Train Unconditional Version

In [12]:
def compute_prior_metrics_conditional_model(model, sampler, train_set, test_props, num_sample = 1000, num_decode = 100):
    '''
    We sample 1000 latent representations z ∼ N (O, I). For each of them we decode 100 times, 
    and calculate the portion of 100,000 decoded results that corresponds to valid Program or SMILES sequences.

    %Valid = #Valid / #Generations
    %Unique = #Unique Valid / #Valid
    %Novel = #Valid - #Valid_in_train_set / #Valid
    MSE = Mse (target_property - achieved_property)^2
    '''

    # TODO: Currently hardcoded for ['LogP']
    
    # Check we have enough condit. values to give to model
    assert num_sample < len(test_props)
    train_set = set(train_set.copy())
    
    test_props = test_props.copy()

    # Get num_sample properties
    props = random.sample(test_props, num_sample)

    n_all_smiles = 0
    n_valid_smiles = 0
    n_valid_in_train = 0
    
    unique_valid = set()

    total_se = 0

    for prop in tqdm(props):
        repeated_prop = torch.tensor([[prop]] * num_decode)
        prop_smiles =  sampler.sample(model, repeated_prop.clone())

        for smi in prop_smiles:
            n_all_smiles += 1
            # Check if valid
            mol = Chem.MolFromSmiles(smi)

            batch_mse = 0

            if mol is not None: # mol is valid
                can_smi = Chem.MolToSmiles(mol, canonical=True)
                n_valid_smiles += 1
                unique_valid.add(can_smi)

                # Compute property
                total_se += ( pc(mol)[0] - prop)**2
                          
                # Check if in train set
                if can_smi in train_set:
                    n_valid_in_train += 1

    # Compute metrics
    validity = n_valid_smiles / n_all_smiles
    uniqueness = len(unique_valid) / n_valid_smiles
    novelty = (n_valid_smiles - n_valid_in_train) / n_valid_smiles
    mse = total_se / n_valid_smiles

    assert n_all_smiles == num_sample * num_decode

    return {'validity' : validity,
            'uniqueness' : uniqueness,
            'novelty' : novelty,
            'MSE': mse}

In [17]:
def smiles_to_tokens(test_smiles, max_seq_len):
    
    tokens = torch.zeros([len(test_smiles), max_seq_len], dtype=torch.long)
    
    for i in range(len(test_smiles)):
        smi = sd.encode(test_smiles[i])
        tokens[i][0] = sd.char_idx[sd.BEGIN]

        for j in range(len(smi)):
            tokens[i][j+1] = sd.char_idx[smi[j]]
            
        tokens[i][len(smi)+1] = sd.char_idx[sd.END]

    return tokens


def compute_reconstruction_vae_model(test_smiles, test_props, vae_model, vae_sampler, n_encodes = 10, n_decodes = 25):
    '''
    test_smiles is array-like of test smiles for reconstruction
    test_props is 

    n_encodes is number of encodings per test_set instance
    n_decodes is number of decodings per latent encoding

    Do one at a time since we're deconding 250x more points

    From SD-VAE paper:
    for each of the structured data in the held-out dataset, we encode it 10 times and decoded (for each encoded latent space representation) 25 times, and report the portion of decoded structures that are the same as the input ones
    '''
    n_same = 0
    n_total = 0
    
    test_props = test_props.copy()
        
    # Normalize properties
    normd_props = torch.tensor([[a] for a in test_props])

    normd_props = vae_model.normalize_prop_scores(normd_props)

    vae_model.eval()
    vae_model.reparam = True


    for smindex in tqdm(range(len(test_smiles))):
        # max_seq_len = model.max_decode_steps
        # enc_b_size = n_encodes
        smile = test_smiles[smindex:smindex+1]
        prop = normd_props[smindex:smindex+1]

        # Convert smiles to tokens
        tokens = smiles_to_tokens(smile, vae_model.max_decode_steps)

        input_tokens = tokens.repeat(n_encodes, 1)
        input_properties = prop.repeat(n_encodes, 1)
        
        # Run through the model
        encodings = vae_model.encoder(input_tokens)[0]
        samp_latents = vae_model.reparameterize(mu = encodings, logvar = torch.tensor(vae_model.eps_std))

        # Repeat latents n_decode times
        # Repeat properties n_decode times
        samp_latents = samp_latents.repeat(n_decodes, 1) 
        repeated_props = input_properties.repeat(n_decodes, 1)

        sampled_smiles = vae_sampler.sample(model=vae_model, properties=repeated_props, random = True, latent_points = samp_latents)

        # Samp latents now contains n_encodings encoded (random latent points)
        # TODO: Do I reparam here? It's the only source of randomness for encodings but not how inference with a vae is typically done
        # feels right for judging the reconstruction loss though

        assert len(sampled_smiles) == n_encodes * n_decodes

        # Now decode probabilitically n_decode times
        counts = Counter(sampled_smiles)
        
        n_same += counts[smile[0]]

        n_total += len(sampled_smiles)

    return n_same / n_total


In [18]:
compute_reconstruction_vae_model(smiles_1k, properties_1k, cvae_model, cvae_sampler, n_encodes = 5, n_decodes = 5) #10 , 25

  0%|          | 0/300 [00:00<?, ?it/s]/home/chrome/Code/MolGen/my_code/./VANILLA_VAE/benchmark_vanilla_vae.py:155: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  properties = torch.tensor(properties).clone()
/home/chrome/Code/MolGen/my_code/./VANILLA_VAE/benchmark_vanilla_vae.py:128: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  latent_points = torch.tensor(latent_points, dtype=torch.float32)
/home/chrome/Code/MolGen/my_code/./VANILLA_VAE/model_vanilla_vae.py:269: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y).float()
1

NameError: name 'n_sn_totalame' is not defined

In [40]:
compute_prior_metrics_conditional_model(cvae_model, cvae_sampler, train_smiles, test_props = test_props, num_sample = 200, num_decode = 5)

  0%|          | 0/200 [00:00<?, ?it/s]

/home/chrome/Code/MolGen/my_code/./VANILLA_VAE/benchmark_vanilla_vae.py:155: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  properties = torch.tensor(properties).clone()
/home/chrome/Code/MolGen/my_code/./VANILLA_VAE/model_vanilla_vae.py:269: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y).float()
100%|██████████| 200/200 [00:12<00:00, 16.32it/s]


{'validity': 0.005,
 'uniqueness': 1.0,
 'novelty': 0.6,
 'MSE': 2.9009181820000043}

In [27]:
# compute_prior_metrics_conditional_model(cvae_model, cvae_sampler, train_smiles, test_props = [0.0 for _ in range(len(test_props)) ], num_sample = 300, num_decode = 5)
compute_prior_metrics_conditional_model(exp_cvae_model, exp_cvae_sampler, train_smiles, test_props = [0.0 for _ in range(len(test_props))], num_sample = 500, num_decode = 5)

  0%|          | 0/300 [00:00<?, ?it/s]

/home/chrome/Code/MolGen/my_code/./VANILLA_VAE/benchmark_vanilla_vae.py:155: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  properties = torch.tensor(properties).clone()
/home/chrome/Code/MolGen/my_code/./VANILLA_VAE/model_vanilla_vae.py:269: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y).float()
100%|██████████| 300/300 [00:18<00:00, 16.03it/s]


{'validity': 0.3453333333333333,
 'uniqueness': 0.6042471042471043,
 'novelty': 0.8359073359073359,
 'MSE': 3.8370004655212395}

In [80]:
compute_prior_metrics_conditional_model(exp_cvae_model, exp_cvae_sampler, train_smiles,  test_props = test_props, num_sample = 200, num_decode = 5)

  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:48<00:00,  4.13it/s]


{'validity': 0.128,
 'uniqueness': 0.6953125,
 'novelty': 0.8671875,
 'MSE': 0.6218614672187499}

In [14]:
# THIS SHOULD ABSOLUTELY NOT WORK!!!
# smiles_1k = random.choices(test_smiles, k=300) # 100
# properties_1k = random.choices(test_props, k=300)

# THIS SHOULD ABSOLUTELY WORK

indices = random.sample(range(len(test_smiles)), 300)
smiles_1k = [test_smiles[i] for i in indices]
properties_1k = [test_props[i] for i in indices]

# cVAE:
#   (Random): 0.47986666666666666
#   (Correct): 0.7353333333333333


# cVAE - POL:
#   (Random) Different indecies gives: 0.3324
#   (Correct) Same indecies gives : 0.4868

In [44]:
compute_reconstruction_vae_model(smiles_1k, properties_1k, cvae_model, cvae_sampler, n_encodes = 5, n_decodes = 5) #10 , 25

100%|██████████| 300/300 [00:52<00:00,  5.67it/s]


0.2812

In [30]:
# properties_1k = [0.0 for _ in range(300)]
compute_reconstruction_vae_model(smiles_1k, properties_1k, cvae_model, cvae_sampler, n_encodes = 5, n_decodes = 5) #10 , 25

100%|██████████| 300/300 [00:50<00:00,  5.88it/s]


0.24386666666666668

### LSTM EVALUATION

In [68]:
# Load SD-LSTM Model
# lstm_params = 'models/LONG_RUNS/ZINC/LSTM/LSTM_final_0.769.pt'
# lstm_definit = 'models/LONG_RUNS/ZINC/LSTM/LSTM_final_0.769.json'

# lstm_definit = 'models/NEW_LONG_RUNS/QM9/LSTM/LSTM_3_1.216.json'
# lstm_params = 'models/NEW_LONG_RUNS/QM9/LSTM/LSTM_3_1.216.pt'

# lstm_definit = 'models/NEW_LONG_RUNS/QM9/LSTM/LSTM_7_1.224.json'
# lstm_params = 'models/NEW_LONG_RUNS/QM9/LSTM/LSTM_7_1.224.pt'

# lstm_definit = 'models/NEW_LONG_RUNS/QM9/LSTM/LSTM_22_1.158.json'
# lstm_params = 'models/NEW_LONG_RUNS/QM9/LSTM/LSTM_22_1.158.pt'

# lstm_definit = 'models/NEW_LONG_RUNS/QM9/LSTM/LSTM_82_1.141.json'
# lstm_params = 'models/NEW_LONG_RUNS/QM9/LSTM/LSTM_82_1.141.pt'

# lstm_definit = 'models/LSTM_QM9/batch_size_64_2/LSTM_20_1.190.json'
# lstm_params = 'models/LSTM_QM9/batch_size_64_2/LSTM_20_1.190.pt'

lstm_definit = '../my_code_upload_old_params/models/LSTM_QM9/batch_size_64_2/LSTM_10_1.211.pt'
lstm_params = '../my_code_upload_old_params/models/LSTM_QM9/batch_size_64_2'

lstm_sampler = FastSampler(device = 'cpu', batch_size=64)
lstm_model = load_rnn_model(
            model_definition= lstm_definit,
            model_weights = lstm_params,
            device = 'cpu',
            )

In [69]:
def compute_metrics_lstm(lstm_sampler, lstm_model, target_props, train_smiles, seq_len = 278):
    # Sample smiles from property scores
    sample_smiles = lstm_sampler.sample(model=lstm_model, properties=target_props, num_to_sample=len(target_props), max_seq_len=seq_len)
    assert len(sample_smiles) == len(target_props)
    n_decodes = 0
    n_valid = 0
    n_valid_in_train = 0
    unique_valid = set()
    total_se = 0.0
    train_set = set(train_smiles)

    for i in range(len(sample_smiles)):
        n_decodes += 1
        smi = sample_smiles[i]
        prop = target_props[i]

        mol = Chem.MolFromSmiles(smi)

        if mol is not None:
            n_valid += 1
            can_smi = Chem.MolToSmiles(mol, canonical=True)
            unique_valid.add(can_smi)
            
            total_se += ( pc(mol)[0] - prop)**2
                          
        # Check if in train set
        if can_smi in train_set:
            n_valid_in_train += 1

    validity = n_valid / n_decodes
    uniqueness = len(unique_valid) / n_valid
    novelty = (n_valid - n_valid_in_train) / n_valid
    mse = total_se / n_valid


    return validity, uniqueness, novelty, mse

In [70]:
target_props = random.choices(test_props, k=300)
target_props = torch.tensor([[a] for a in target_props])

In [71]:
compute_metrics_lstm(lstm_sampler, lstm_model, train_smiles=train_smiles, target_props=target_props)

(1.0, 0.24, 0.043333333333333335, tensor([0.1025]))

## Property Error Baseline

In [13]:
def mean_squared_error(list1, list2):
    if len(list1) != len(list2):
        raise ValueError("Lists must have the same length")
    
    mse = sum((a - b) ** 2 for a, b in zip(list1, list2)) / len(list1)
    return mse

In [14]:
propsA = test_props
propsB = random.sample(test_props, len(test_props))

mse = mean_squared_error(propsA, propsB)
print(f'Baseline MSE is: {mse}')

Baseline MSE is: 2.003641019857367


In [16]:
mean = np.mean(test_props) 
propsMean = [mean for _ in range(len(test_props))]

mse = mean_squared_error(propsMean, propsA)
print(f'Baseline MSE is: {mse}')

Baseline MSE is: 0.9960336400890705


0.286706302